In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 2/ Import Data

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
test.info()

### Combine Trait and Test

In [ ]:
all_data = [train, test]

### Identify Missing Values

In [ ]:
pd.concat([train, test], axis = 0).isnull().sum()
# No Missing Values

## Numerical Preprocessing
- Missing
- Outlier
- Tranformation

In [ ]:
numerical_features = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10']
#categorical_features = ['cat0', 'cat1', 'cat2']

### NO Missing Treatement

### Outlier

In [ ]:
import seaborn as sns
sns.boxplot(train['cont9']) 

In [ ]:
# Write a loop to detect and impute Outliers
for num_var in numerical_features:
  Q1 = train[num_var].quantile(0.25)
  Q3 = train[num_var].quantile(0.75)

  IQR = Q3-Q1

  Lower_Whisker = Q1 - 1.5*IQR
  Upper_Whisker = Q3 + 1.5*IQR

  for dataset in all_data:
    dataset[num_var] = dataset[num_var].apply(lambda x : Upper_Whisker if x > Upper_Whisker else x)
    dataset[num_var] = dataset[num_var].apply(lambda x : Lower_Whisker if x < Lower_Whisker else x)

## No Tranformation

## 7/ Build Model

In [ ]:
# Get X and y Variables for your Dataset
X = train.drop(['target','id', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18'], axis =  1)
y = train['target']

In [ ]:
# Get Dummy Variables
X = pd.get_dummies(X)  # Keep only n - 1 Variables - Use "drop_first = True"

In [ ]:
X.head()

In [ ]:
#Import Logistic Regression Library
from sklearn.linear_model import LogisticRegression

#Initialize LR
clf = LogisticRegression(max_iter = 1000)

In [ ]:
# Fit Model
model = clf.fit(X, y)

## 7/ Evaluation

In [ ]:
# Predict on Train Data
y_pred = model.predict(X)

In [ ]:
# Import metrics library
from sklearn.metrics import accuracy_score

In [ ]:
# get Actual "y" variables (use "y_true" as variable)
y_true = train['target']

In [ ]:
print(f" Train Accuracy : {accuracy_score(y_true, y_pred):0.1%}")

## Predict

In [ ]:
# Get all the X Variables from the Test Dataset
X_test = test.drop(['id', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18'], axis =  1)

In [ ]:
#Covert to Dummies
X_test = pd.get_dummies(X_test)

In [ ]:
X_test_prep = model.predict(X_test)

In [ ]:
submission = pd.DataFrame({
    'id' : test['id'],
    'target' : X_test_prep
})

In [ ]:
submission.to_csv('Submission_log_reg1.csv', index = False)